# Data Re-Uploading QCNN_TTN

This demo uses 4-qubit Quantum Convolutional Neural Network (QCNN) to see how pre-training the quantum embedding can be helpful for training a parameterized QML circuits for classfication tasks.

If you are interested in detailed analysis of pre-training on 8-qubit device, check out "/Result/earlystop 10 experiments/" folder.

If you are interested in the details about the QCNN used in this demo, check out https://arxiv.org/pdf/2108.00661.pdf.


In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/tak/Github/QEmbedding/')
import torch
from torch import nn
import data
import embedding

2024-04-02 12:07:07.498283: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


IBMQ Fake provider

In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

/tmp/ipykernel_275370/1611809002.py:8: DeprecationWarning: All fake backend instances based on real device snapshots (`FakeVigo`,`FakeSherbrooke`,...) have been migrated to the `qiskit_ibm_runtime` package. These classes are deprecated as of qiskit 0.46.0 and will be removed in qiskit 1.0.0. To migrate your code, run `pip install qiskit-ibm-runtime` and use `from qiskit_ibm_runtime.fake_provider import FakeExample` instead of `from qiskit.providers.fake_provider import FakeExample`. If you are using a custom fake backend implementation, you don't need to take any action.
  noisy = FakeJakarta()


# Part1: Noisy Model1 & Model 2

Load the dataset for Model1 and Model2

In [3]:
dev = qml.device('default.qubit', wires=4)
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('fashion', feature_reduction=feature_reduction, classes=classes)

2024-04-02 12:07:09.019040: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Part1: Pre-training the embedding

In [4]:
steps = 1000
learning_rate = 0.01
batch_size = 25
ansatz = 'TTN'

def data_reuploading_ansatz(params): #8 params
    for i in range(4):
        qml.RY(params[i], wires=i)
    for i in range(3):
        qml.IsingZZ(params[i+4], wires=[i,i+1])
    qml.IsingZZ(params[7], wires=[3,0])

def data_reuploading_embedding(num_layers, params, x):
    for l in range(num_layers):
        data_reuploading_ansatz(params[8 * l : 8 * (l + 1)])
        embedding.Noisy_Four_QuantumEmbedding1(x)


@qml.qnode(dev_fake)
def data_reuploading_QCNN_classifier(num_layers, params, x):
    data_reuploading_embedding(num_layers, params, x)
    embedding.Noisy_QCNN_four(params[8 * num_layers: 8 * num_layers + 4], ansatz)
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, num_layers):
    preds = [data_reuploading_QCNN_classifier(num_layers, weights, x) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, num_layers):

    if ansatz == 'SU4':
        num_weights = (num_layers * 8) + 30
    elif ansatz == 'TTN':
        num_weights = (num_layers * 8) + 4

    weights = np.random.random(num_weights, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, num_layers),
                                                     weights)
        loss_history.append(cost_new)
        if it % 200 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

In [5]:

Loss_histories_L1, weights_L1, Loss_histories_L2, weights_L2, Loss_histories_L3, weights_L3 = [], [], [], [], [], []
for i in range(5):
    #loss_L1, weight_L1 = circuit_training(X_train, Y_train, 1)
    loss_L2, weight_L2 = circuit_training(X_train, Y_train, 2)
    #loss_L3, weight_L3 = circuit_training(X_train, Y_train, 3)

    #Loss_histories_L1.append(loss_L1)
    Loss_histories_L2.append(loss_L2)
    #Loss_histories_L3.append(loss_L3)

    #weights_L1.append(weight_L1)
    weights_L2.append(weight_L2)
    #weights_L3.append(weight_L3)

f = open('data_reuploading_weights_and_loss_fashion.txt', 'a')

for i in range(5):
    f.write(f'Loss History L=2 {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_L2[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights L=2 {i + 1}:')
    f.write('\n')
    f.write(str(weights_L2[i]))
f.close()



iteration:  0  cost:  0.4933984375
iteration:  200  cost:  0.488125
iteration:  400  cost:  0.46078125


KeyboardInterrupt: 